# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 19 2023 9:25AM,261392,10,CTF0011730,"Citieffe, Inc.",Released
1,Apr 19 2023 9:24AM,261391,10,MSP220908,"Methapharm, Inc.",Released
2,Apr 19 2023 9:24AM,261391,10,MSP220911,"Methapharm, Inc.",Released
3,Apr 19 2023 9:24AM,261391,10,MSP220904,"Methapharm, Inc.",Released
4,Apr 19 2023 9:23AM,261390,19,9395405,"Emersa Waterbox, LLC",Released
5,Apr 19 2023 9:22AM,261389,10,MSP220909,"Methapharm, Inc.",Released
6,Apr 19 2023 9:21AM,261388,16,9394883,Sartorius Bioprocess Solutions,Released
7,Apr 19 2023 9:09AM,261387,10,38102,"Digital Brands, LLC",Released
8,Apr 19 2023 9:09AM,261387,10,38103,"Digital Brands, LLC",Released
9,Apr 19 2023 9:09AM,261387,10,38104,"Digital Brands, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
19,261388,Released,1
20,261389,Released,1
21,261390,Released,1
22,261391,Released,3
23,261392,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
261388,NaN,1.0
261389,NaN,1.0
261390,NaN,1.0
261391,NaN,3.0
261392,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
260869,0.0,33.0
260910,7.0,18.0
261117,0.0,10.0
261300,1.0,0.0
261331,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
260869,0,33
260910,7,18
261117,0,10
261300,1,0
261331,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,260869,0,33
1,260910,7,18
2,261117,0,10
3,261300,1,0
4,261331,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,260869,,33
1,260910,7,18
2,261117,,10
3,261300,1,
4,261331,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 19 2023 9:25AM,261392,10,"Citieffe, Inc."
1,Apr 19 2023 9:24AM,261391,10,"Methapharm, Inc."
4,Apr 19 2023 9:23AM,261390,19,"Emersa Waterbox, LLC"
5,Apr 19 2023 9:22AM,261389,10,"Methapharm, Inc."
6,Apr 19 2023 9:21AM,261388,16,Sartorius Bioprocess Solutions
7,Apr 19 2023 9:09AM,261387,10,"Digital Brands, LLC"
15,Apr 19 2023 8:57AM,261386,10,"Nextsource Biotechnology, LLC"
16,Apr 19 2023 8:56AM,261385,10,Bio-PRF
26,Apr 19 2023 8:50AM,261384,10,Emerginnova
27,Apr 19 2023 8:45AM,261383,10,Hush Hush


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Apr 19 2023 9:25AM,261392,10,"Citieffe, Inc.",,1
1,Apr 19 2023 9:24AM,261391,10,"Methapharm, Inc.",,3
2,Apr 19 2023 9:23AM,261390,19,"Emersa Waterbox, LLC",,1
3,Apr 19 2023 9:22AM,261389,10,"Methapharm, Inc.",,1
4,Apr 19 2023 9:21AM,261388,16,Sartorius Bioprocess Solutions,,1
5,Apr 19 2023 9:09AM,261387,10,"Digital Brands, LLC",,8
6,Apr 19 2023 8:57AM,261386,10,"Nextsource Biotechnology, LLC",,1
7,Apr 19 2023 8:56AM,261385,10,Bio-PRF,,10
8,Apr 19 2023 8:50AM,261384,10,Emerginnova,,1
9,Apr 19 2023 8:45AM,261383,10,Hush Hush,,8


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 19 2023 9:25AM,261392,10,"Citieffe, Inc.",1,
1,Apr 19 2023 9:24AM,261391,10,"Methapharm, Inc.",3,
2,Apr 19 2023 9:23AM,261390,19,"Emersa Waterbox, LLC",1,
3,Apr 19 2023 9:22AM,261389,10,"Methapharm, Inc.",1,
4,Apr 19 2023 9:21AM,261388,16,Sartorius Bioprocess Solutions,1,
5,Apr 19 2023 9:09AM,261387,10,"Digital Brands, LLC",8,
6,Apr 19 2023 8:57AM,261386,10,"Nextsource Biotechnology, LLC",1,
7,Apr 19 2023 8:56AM,261385,10,Bio-PRF,10,
8,Apr 19 2023 8:50AM,261384,10,Emerginnova,1,
9,Apr 19 2023 8:45AM,261383,10,Hush Hush,8,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 19 2023 9:25AM,261392,10,"Citieffe, Inc.",1,
1,Apr 19 2023 9:24AM,261391,10,"Methapharm, Inc.",3,
2,Apr 19 2023 9:23AM,261390,19,"Emersa Waterbox, LLC",1,
3,Apr 19 2023 9:22AM,261389,10,"Methapharm, Inc.",1,
4,Apr 19 2023 9:21AM,261388,16,Sartorius Bioprocess Solutions,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Apr 19 2023 9:25AM,261392,10,"Citieffe, Inc.",1.0,NaN
1,Apr 19 2023 9:24AM,261391,10,"Methapharm, Inc.",3.0,NaN
2,Apr 19 2023 9:23AM,261390,19,"Emersa Waterbox, LLC",1.0,NaN
3,Apr 19 2023 9:22AM,261389,10,"Methapharm, Inc.",1.0,NaN
4,Apr 19 2023 9:21AM,261388,16,Sartorius Bioprocess Solutions,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 19 2023 9:25AM,261392,10,"Citieffe, Inc.",1.0,0.0
1,Apr 19 2023 9:24AM,261391,10,"Methapharm, Inc.",3.0,0.0
2,Apr 19 2023 9:23AM,261390,19,"Emersa Waterbox, LLC",1.0,0.0
3,Apr 19 2023 9:22AM,261389,10,"Methapharm, Inc.",1.0,0.0
4,Apr 19 2023 9:21AM,261388,16,Sartorius Bioprocess Solutions,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3136542,37.0,1.0
12,261368,1.0,0.0
16,261388,1.0,0.0
19,1306822,31.0,0.0
20,1044196,61.0,8.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3136542,37.0,1.0
1,12,261368,1.0,0.0
2,16,261388,1.0,0.0
3,19,1306822,31.0,0.0
4,20,1044196,61.0,8.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,37.0,1.0
1,12,1.0,0.0
2,16,1.0,0.0
3,19,31.0,0.0
4,20,61.0,8.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,37.0
1,12,Released,1.0
2,16,Released,1.0
3,19,Released,31.0
4,20,Released,61.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20
Status,,,,,
Executing,1.0,0.0,0.0,0.0,8.0
Released,37.0,1.0,1.0,31.0,61.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20
0,Executing,1.0,0.0,0.0,0.0,8.0
1,Released,37.0,1.0,1.0,31.0,61.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20
0,Executing,1.0,0.0,0.0,0.0,8.0
1,Released,37.0,1.0,1.0,31.0,61.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()